In [1]:
# Getting the required libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models
from keras.losses import categorical_crossentropy
from keras.wrappers.scikit_learn import KerasClassifier
#
from sklearn.model_selection import train_test_split,cross_validate,cross_val_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix
from scipy import stats
#
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
dataset = '/content/drive/My Drive/Concatenated.csv'
df = pd.read_csv(dataset)
df.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(163840, 17)

In [2]:
X = df.drop(labels='Label',axis=1)
Y = df['Label']
D = X.shape
#final_acc = []
# Buildding a 1D CNN model
model = models.Sequential()
model.add(layers.Conv1D(filters= 128, kernel_size = 3, strides=1,
                        padding='same',activation='relu',
                        input_shape=(D[1],1)))
model.add(layers.Conv1D(filters = 128, kernel_size = 3, strides = 1,
                        activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
#model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [ ]:
# Create splits
from sklearn.model_selection import StratifiedKFold
import statistics
sen = []
spe = []
acc = []
auc = []
senM = []
speM = []
accM = []
aucM = []
win = 256
kf = StratifiedKFold(n_splits=10, shuffle= True, random_state=100)
splits = kf.split(X,Y)
for train_index, val_index in splits:
  Y_train = Y[train_index]
  Y_test = Y[val_index]
  X_train = X.values[train_index,]
  X_test = X.values[val_index,]
  #
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  trDim = X_train.shape
  X_train = X_train.reshape(trDim[0],trDim[1],1)
  dim = X_test.shape
  X_test = X_test.reshape(dim[0],dim[1],1)
  hist = model.fit(X_train,Y_train,epochs=15,verbose=0, 
                   validation_freq = 10) 
  score,tacc = model.evaluate(X_test,Y_test)
  prob = model.predict_proba(X_test)
  Y_predict = model.predict_classes(X_test)
  C = confusion_matrix(Y_test,Y_predict)
  TP = C[1,1]
  FN= C[1,0]
  FP= C[0,1]
  TN= C[0,0]
  sen.append(TP /(TP + FN))
  spe.append(TN /(TN + FP))
  acc.append(tacc)
  auc.append(roc_auc_score(Y_test,prob))
  # MAjority voting
  L = len(Y_test)
  Y_test = np.array(Y_test)
  Y_test = Y_test[np.arange(0,L-1,win)]
  LL = np.int(L/win)
  prob1 = []
  Pred1 = []
  for i in range(LL):
    #prob = np.array(prob)
    ini = i*win; fin = win*(i+1)
    prob1.append(np.mean(prob[ini:fin])) 
    tmp = stats.mode(Y_predict[ini:fin])
    tmp = tmp.mode
    Pred1.append(tmp.item())
  # majority voting end
  C1 = confusion_matrix(Y_test,Pred1)
  TP1 = C1[1,1]
  FN1 = C1[1,0]
  FP1 = C1[0,1]
  TN1 = C1[0,0]
  senM.append(TP1 /(TP1 + FN1))
  speM.append(TN1 /(TN1 + FP1))
  accM.append((TP1+TN1)/(TN1+TP1+FN1+FP1))
  aucM.append(roc_auc_score(Y_test,prob1))
  print(C1)

512/512 [==============================] - 1s 2ms/step - loss: 0.2738 - accuracy: 0.8783


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[32  0]
 [ 0 32]]


In [ ]:
noEnsResults = pd.DataFrame(np.matrix.transpose(np.matrix([sen,spe,acc,auc])),
             columns=['Sensitvity','Specificity','Accuracy','AUCROC'])
EnsemResults = pd.DataFrame(np.matrix.transpose(np.matrix([senM,speM,accM,aucM])),
             columns=['Sensitvity','Specificity','Accuracy','AUCROC'])
noEnsResults.to_csv("NoEnsembleCNNResults.csv")
EnsemResults.to_csv("EnsembleCNNResults.csv")